<a href="https://colab.research.google.com/github/Pankaj9824/Infoya-AI/blob/Pankaj/QE_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pandas as pd
import re
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/QE/Training_2.csv', encoding='latin-1')

In [ ]:
X = dataset['input']
y = dataset['expected_outcomes']

In [ ]:
print(X.shape)

(29,)


In [ ]:
print(X[2])

User should able to place order with paypal user information not allowed to edit returning from paypal


In [ ]:
print(y.shape)

(29,)


In [ ]:
print(y[2])

Then the user enters text "7001" in textbox with placeholder "Postal Code, City, or Store Number"
    Then the user clicks on the webelement with html tag "title" as "Search"
    Then the user waits for webelement "hdca-store-list-item__column-store" with html attribute "class" to be visible
    Then the user click on "Select"
    Then the user enters text sku in textbox with placeholder "What can we help you find?"
    Then the user clicks on the webelement with html tag "class" as "acl-action-button"
    Then the user waits for text " Delivery " to be visible
    Then the user add product to cart
    Then the user click on "View Cart"
    Then the user waits for webelement "hdca-cart__header-title" with html attribute "class" to be visible
    Then the user selects the radio button "Standard Shipping:"
    Then the user validates if the 'Standard Shipping:' radio button at index "1" is selected
    Then the user click on "Checkout Now"
    Then the user waits for text "Secure Checkou

In [ ]:
print(type(X),type(y))

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [ ]:
# Preprocessing steps
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))

    tokens = nltk.word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

# Preprocess both input and expected_outcomes columns
X = X.apply(preprocess_text)
y = y.apply(preprocess_text)

# Tokenization
tokenizer = Tokenizer(filters='', split=' ')  # Split on spaces to maintain special tokens

tokenizer.fit_on_texts(X)
input_sequences = tokenizer.texts_to_sequences(X)
input_max_sequence_len = max([len(seq) for seq in input_sequences])
input_padded_sequences = pad_sequences(input_sequences, maxlen=input_max_sequence_len, padding='pre')

tokenizer.fit_on_texts(y)
output_sequences = tokenizer.texts_to_sequences(y)
output_max_sequence_len = max([len(seq) for seq in output_sequences])
output_padded_sequences = pad_sequences(output_sequences, maxlen=output_max_sequence_len, padding='pre')

tokenizer.word_index["start"] = len(tokenizer.word_index) + 1
tokenizer.word_index["end"] = len(tokenizer.word_index) + 2

# Split into predictors and labels
input_predictors = input_padded_sequences[:, :-1]
output_predictors = output_padded_sequences[:, :-1]
output_labels = output_padded_sequences[:, 1:]  # Shifted by one position

In [ ]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# # Vocabulary size
# vocab_size = len(tokenizer.word_index) + 1

# # Input sequence
# input_sequence_len = input_max_sequence_len - 1  # Minus 1 to exclude the last token
# input_sequence = Input(shape=(input_sequence_len,))

# # Embedding layer
# embedding_dim = 100  # Adjust as needed
# embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)

# # Encoder LSTM
# encoder_units = 256  # Adjust as needed
# encoder_lstm = LSTM(encoder_units, return_state=True)
# _, state_h, state_c = encoder_lstm(embedding_layer)

# # Decoder input
# decoder_input_sequence = Input(shape=(output_max_sequence_len - 1,))  # Minus 1 for shift

# # Embedding layer for decoder input
# decoder_embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_input_sequence)

# # Decoder LSTM with initial states from encoder
# decoder_lstm = LSTM(encoder_units, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_embedding_layer, initial_state=[state_h, state_c])

# # Output layer
# output_layer = Dense(vocab_size, activation='softmax')
# output = output_layer(decoder_outputs)

# # Define the model
# model = Model([input_sequence, decoder_input_sequence], output)

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Print the model summary
# model.summary()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Attention, Dense
from tensorflow.keras.models import Model

# Define input sequence length and output sequence length
"""These variables define the lengths of the input and output sequences.
 The -1 adjustment is applied to account for the fact that the model processes sequences one time step at a time,
 and the last token is used as the start token for the decoder or the "shift" for target sequences."""
input_seq_len = input_max_sequence_len - 1
output_seq_len = output_max_sequence_len - 1

# Define vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Define embedding dimension
"""embedding_dim specifies the dimensionality of the word embeddings. Each word in the vocabulary will be represented as a dense vector of this dimension."""
embedding_dim = 128

# Define the model inputs
""" These lines define the input layers for the encoder and decoder.
 Each layer expects sequences of integers with lengths equal to input_seq_len and output_seq_len, respectively. """
encoder_input = Input(shape=(input_seq_len,))
decoder_input = Input(shape=(output_seq_len,))

# Create an embedding layer for both encoder and decoder inputs
""" An embedding layer is created using the Embedding class from Keras.
This layer maps the input integer sequences to dense vectors of length embedding_dim. Both the encoder and decoder inputs are embedded using the same layer."""
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# Apply embedding layer to inputs
encoder_embedding = embedding_layer(encoder_input)
decoder_embedding = embedding_layer(decoder_input)

# Create the encoder using a multi-head self-attention mechanism
""" Here, a bidirectional LSTM layer is used for the encoder.
 The bidirectional layer processes the input sequence in both the forward and backward directions.
  The LSTM returns both the sequences of outputs (encoder_outputs) and the hidden states (forward_h, backward_h) and cell states (forward_c, backward_c) of the LSTM units."""
encoder_outputs, forward_h, forward_c, backward_h, backward_c = keras.layers.Bidirectional(
    keras.layers.LSTM(embedding_dim, return_sequences=True, return_state=True)
)(encoder_embedding)

# Concatenate the forward and backward LSTM outputs
encoder_state_h = keras.layers.Concatenate()([forward_h, backward_h])
encoder_state_c = keras.layers.Concatenate()([forward_c, backward_c])

# Define the decoder LSTM
""" A unidirectional LSTM layer is used for the decoder.
This layer takes the embedded decoder input and the initial states (encoder_state_h and encoder_state_c) as inputs and returns sequences of outputs (decoder_outputs) as well as the hidden and cell states of the LSTM."""
decoder_lstm = keras.layers.LSTM(embedding_dim * 2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_state_h, encoder_state_c])

# Apply attention mechanism to the decoder outputs
""" An attention mechanism is applied to align the decoder outputs with relevant parts of the encoder outputs.
This helps the decoder focus on different parts of the input sequence while generating each output token."""
attention_layer = Attention()([decoder_outputs, encoder_outputs])

# Concatenate attention output and decoder LSTM output
""" The outputs from the decoder LSTM and the attention mechanism are concatenated to create a combined context, which is then used as input to the output layer."""
decoder_combined_context = keras.layers.Concatenate()([decoder_outputs, attention_layer])

# Create the output layer (dense layer)
""" A dense (fully connected) layer with a softmax activation function is used as the output layer.
 It generates a probability distribution over the vocabulary for each position in the output sequence."""
output_layer = Dense(vocab_size, activation='softmax')
output = output_layer(decoder_combined_context)

# Define the model
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])

# Compile the model
""" The model is compiled with the Adam optimizer and the sparse categorical cross-entropy loss function. Accuracy is used as a metric for evaluation """
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1117)]       0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 22)]         0           []                               
                                                                                                  
 embedding (Embedding)          multiple             83072       ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  [(None, 22, 256),    263168      ['embedding[0][0]']          

In [ ]:
# Training the Model

# Define the input and output data
input_data = [input_predictors, output_predictors]
output_data = output_labels

# Set the number of epochs and batch size
epochs = 20
batch_size = 64

# Train the model
model.fit(input_data, output_data, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/20
1/1 [==============================] - 14s 14s/step - loss: 6.4730 - accuracy: 2.3354e-04 - val_loss: 6.4189 - val_accuracy: 0.5109
Epoch 2/20
1/1 [==============================] - 6s 6s/step - loss: 6.4251 - accuracy: 0.4696 - val_loss: 6.3508 - val_accuracy: 0.5109
Epoch 3/20
1/1 [==============================] - 6s 6s/step - loss: 6.3646 - accuracy: 0.4696 - val_loss: 6.2411 - val_accuracy: 0.5109
Epoch 4/20
1/1 [==============================] - 5s 5s/step - loss: 6.2661 - accuracy: 0.4696 - val_loss: 5.9791 - val_accuracy: 0.5109
Epoch 5/20
1/1 [==============================] - 7s 7s/step - loss: 6.0280 - accuracy: 0.4696 - val_loss: 5.1433 - val_accuracy: 0.5109
Epoch 6/20
1/1 [==============================] - 5s 5s/step - loss: 5.2610 - accuracy: 0.4696 - val_loss: 4.3347 - val_accuracy: 0.5109
Epoch 7/20
1/1 [==============================] - 6s 6s/step - loss: 4.4924 - accuracy: 0.4696 - val_loss: 3.8277 - val_accuracy: 0.5109
Epoch 8/20
1/1 [===================

In [ ]:
# Prepare the input scenario
input_scenario = "User should able to place order with paypal user information not allowed to edit returning from paypal"

# Tokenizing and preprocessing the input
preprocessed_input = preprocess_text(input_scenario)
input_sequence = tokenizer.texts_to_sequences([preprocessed_input])
input_sequence = pad_sequences(input_sequence, maxlen=input_max_sequence_len - 1, padding='pre')

# Initializing the decoder with a start token
start_token = "start"
target_seq = tokenizer.texts_to_sequences([start_token])
target_seq = pad_sequences(target_seq, maxlen=output_max_sequence_len - 1, padding='pre')

# Generate predictions
output_sequence = []
for _ in range(output_max_sequence_len - 1):  # Minus 1 for start token
    predicted_output = model.predict([input_sequence, target_seq])
    #print(predicted_output)
    predicted_token_id = np.argmax(predicted_output[0, -1, :])  # Get the index of the highest probability token
    print(predicted_token_id)
    output_sequence.append(predicted_token_id)
    #print(tokenizer.sequences_to_texts([output_sequence]))
    if predicted_token_id == tokenizer.word_index["end"]:
        break
    target_seq[0, -1] = predicted_token_id

# Convert predicted token IDs to words
predicted_output = tokenizer.sequences_to_texts([output_sequence])[0]


1/1 [==============================] - 1s 1s/step
0
1/1 [==============================] - 0s 207ms/step
0
1/1 [==============================] - 0s 204ms/step
0
1/1 [==============================] - 0s 217ms/step
0
1/1 [==============================] - 0s 218ms/step
0
1/1 [==============================] - 0s 232ms/step
0
1/1 [==============================] - 0s 215ms/step
0
1/1 [==============================] - 0s 206ms/step
0
1/1 [==============================] - 0s 212ms/step
0
1/1 [==============================] - 0s 207ms/step
0
1/1 [==============================] - 0s 219ms/step
0
1/1 [==============================] - 0s 201ms/step
0
1/1 [==============================] - 0s 214ms/step
0
1/1 [==============================] - 0s 210ms/step
0
1/1 [==============================] - 0s 209ms/step
0
1/1 [==============================] - 0s 227ms/step
0
1/1 [==============================] - 0s 211ms/step
0
1/1 [==============================] - 0s 212ms/step
0
1/1 [========

In [ ]:
print(predicted_output)
print(output_sequence)
print(len(output_sequence))


1117


In [ ]:
print(tokenizer.word_index)

{'the': 1, 'user': 2, 'then': 3, 'with': 4, 'visible': 5, 'html': 6, 'webelement': 7, 'text': 8, 'if': 9, 'validates': 10, 'is': 11, 'class': 12, 'attribute': 13, 'a': 14, 'on': 15, 'click': 16, 'to': 17, 'in': 18, 'for': 19, 'enters': 20, 'be': 21, 'wait': 22, 'textbox': 23, 'tag': 24, 'placeholder': 25, 'cart': 26, 'city': 27, 'order': 28, 'checkout': 29, 'store': 30, 'pro': 31, 'email': 32, 'number': 33, 'address': 34, 'postal': 35, 'code': 36, 'ha': 37, 'local': 38, 'continue': 39, 'postalcode': 40, 'province': 41, 'or': 42, 'add': 43, 'id': 44, 'product': 45, 'your': 46, 'not': 47, 'title': 48, 'at': 49, 'delivery': 50, 'name': 51, 'search': 52, 'phone': 53, 'sku': 54, 'what': 55, 'select': 56, 'card': 57, 'hdcastorelistitemcolumnstore': 58, 'index': 59, 'me': 60, 'you': 61, 'aclactionbutton': 62, 'should': 63, 'able': 64, 'firstname': 65, 'lastname': 66, 'hdcacartheadertitle': 67, 'example': 68, 'button': 69, 'connect': 70, 'can': 71, 'we': 72, 'help': 73, 'find': 74, 'hdcaproduc

In [ ]:
sample_input = "User should able to place order with paypal user information..."
preprocessed_sample = preprocess_text(sample_input)
tokenized_sample = tokenizer.texts_to_sequences([preprocessed_sample])
print(tokenized_sample)

[[2, 63, 64, 17, 80, 28, 4, 138, 2, 218]]


In [ ]:
vocab_size = len(tokenizer.word_index)
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 648


In [ ]:
print("Token ID for 'start':", tokenizer.word_index["start"])
print("Token ID for 'end':", tokenizer.word_index["end"])

Token ID for 'start': 647
Token ID for 'end': 649
